In [1]:
# https://keras.io/
!pip install -q keras
import keras

Using TensorFlow backend.


In [0]:
import keras
from keras.datasets import cifar10
from keras.models import Model, Sequential
from keras.layers import Dense, Dropout, Flatten, Input, AveragePooling2D, merge, Activation
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization
from keras.layers import Concatenate
from keras.optimizers import Adam,RMSprop

In [0]:
# this part will prevent tensorflow to allocate all the avaliable GPU Memory
# backend
import tensorflow as tf
from keras import backend as k

# Don't pre-allocate memory; allocate as-needed
config = tf.ConfigProto()
config.gpu_options.allow_growth = True

# Create a session with the above options specified.
k.tensorflow_backend.set_session(tf.Session(config=config))

In [0]:
# Hyperparameters
batch_size = 128
num_classes = 10
epochs = 50
l = 40
num_filter = 24
compression = 0.6
dropout_rate = 0.2

In [5]:
# Load CIFAR10 Data
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
img_height, img_width, channel = x_train.shape[1],x_train.shape[2],x_train.shape[3]

# convert to one hot encoing 
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

170500096/170498071 [==============================] - 34s 0us/step


In [0]:
# Dense Block
def add_denseblock(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    temp = input
    for _ in range(l):
        BatchNorm = BatchNormalization()(temp)
        relu = Activation('relu')(BatchNorm)
        Conv2D_3_3 = Conv2D(int(num_filter*compression), (3,3), use_bias=False ,padding='same')(relu)
        if dropout_rate>0:
          Conv2D_3_3 = Dropout(dropout_rate)(Conv2D_3_3)
        concat = Concatenate(axis=-1)([temp,Conv2D_3_3])
        
        temp = concat
        
    return temp

In [0]:
def add_transition(input, num_filter = 24, dropout_rate = 0.2):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    Conv2D_BottleNeck = Conv2D(int(num_filter*compression), (1,1), use_bias=False ,padding='same')(relu)
    if dropout_rate>0:
      Conv2D_BottleNeck = Dropout(dropout_rate)(Conv2D_BottleNeck)
    avg = AveragePooling2D(pool_size=(2,2))(Conv2D_BottleNeck)
    
    return avg

In [0]:
def output_layer(input):
    global compression
    BatchNorm = BatchNormalization()(input)
    relu = Activation('relu')(BatchNorm)
    AvgPooling = AveragePooling2D(pool_size=(2,2))(relu)
    flat = Flatten()(AvgPooling)
    output = Dense(num_classes, activation='softmax')(flat)
    
    return output

In [0]:
num_filter = 24
dropout_rate = 0.2
l = 12
input = Input(shape=(img_height, img_width, channel,))
First_Conv2D = Conv2D(num_filter, (3,3), use_bias=False ,padding='same')(input)

First_Block = add_denseblock(First_Conv2D, num_filter, dropout_rate)
First_Transition = add_transition(First_Block, num_filter, dropout_rate)

Second_Block = add_denseblock(First_Transition, num_filter, dropout_rate)
Second_Transition = add_transition(Second_Block, num_filter, dropout_rate)

Third_Block = add_denseblock(Second_Transition, num_filter, dropout_rate)
Third_Transition = add_transition(Third_Block, num_filter, dropout_rate)

Last_Block = add_denseblock(Third_Transition,  num_filter, dropout_rate)
output = output_layer(Last_Block)


In [10]:
model = Model(inputs=[input], outputs=[output])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 32, 32, 3)    0                                            
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 32, 32, 24)   648         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization_1 (BatchNor (None, 32, 32, 24)   96          conv2d_1[0][0]                   
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 32, 32, 24)   0           batch_normalization_1[0][0]      
__________________________________________________________________________________________________
conv2d_2 (

In [0]:
#Data Augmentation
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rotation_range=15,width_shift_range=0.1,
                             height_shift_range=0.1,horizontal_flip=True)
datagen.fit(x_train)

In [0]:
# determine Loss function and Optimizer
model.compile(loss='categorical_crossentropy',
              optimizer=RMSprop(),
              metrics=['accuracy'])

In [16]:
import os
!ls

datalab


In [17]:
os.mkdir('drive')
!ls

datalab  drive


In [0]:
#Creating Checkpoints after every epoch

from keras.callbacks import ModelCheckpoint

filepath="May16-weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1,
                             save_best_only=False, save_weights_only=True,
                             mode='auto', period=1)
callbacks_list = [checkpoint]

In [0]:

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] / batch_size,epochs=epochs,verbose=1,validation_data=(x_test,y_test),callbacks=callbacks_list)

Epoch 1/50
391/390 [==============================] - 288s 736ms/step - loss: 1.6060 - acc: 0.4044 - val_loss: 1.5288 - val_acc: 0.4842

Epoch 00001: saving model to May16-weights-01-0.48.hdf5
Epoch 2/50
223/390 [================>.............] - ETA: 1:50 - loss: 1.2727 - acc: 0.5372

391/390 [==============================] - 274s 700ms/step - loss: 1.2263 - acc: 0.5531 - val_loss: 2.5152 - val_acc: 0.4160

Epoch 00002: saving model to May16-weights-02-0.42.hdf5
Epoch 3/50
313/390 [=======================>......] - ETA: 50s - loss: 1.0809 - acc: 0.6108

391/390 [==============================] - 274s 700ms/step - loss: 1.0676 - acc: 0.6158 - val_loss: 1.4151 - val_acc: 0.5708

Epoch 00003: saving model to May16-weights-03-0.57.hdf5
Epoch 4/50
340/390 [=========================>....] - ETA: 33s - loss: 0.9684 - acc: 0.6506

391/390 [==============================] - 273s 699ms/step - loss: 0.9638 - acc: 0.6531 - val_loss: 3.2494 - val_acc: 0.4097

Epoch 00004: saving model to May16-weights-04-0.41.hdf5
Epoch 5/50
348/390 [=========================>....] - ETA: 28s - loss: 0.8967 - acc: 0.6779

391/390 [==============================] - 274s 700ms/step - loss: 0.8919 - acc: 0.6796 - val_loss: 1.5504 - val_acc: 0.5777

Epoch 00005: saving model to May16-weights-05-0.58.hdf5
Epoch 6/50
350/390 [=========================>....] - ETA: 26s - loss: 0.8401 - acc: 0.7016

...

391/390 [==============================] - 274s 701ms/step - loss: 0.3991 - acc: 0.8602 - val_loss: 0.5768 - val_acc: 0.8292

Epoch 00040: saving model to May16-weights-40-0.83.hdf5
Epoch 41/50
350/390 [=========================>....] - ETA: 26s - loss: 0.3930 - acc: 0.8637

391/390 [==============================] - 274s 700ms/step - loss: 0.3941 - acc: 0.8631 - val_loss: 0.7642 - val_acc: 0.7929

Epoch 00041: saving model to May16-weights-41-0.79.hdf5
Epoch 42/50
350/390 [=========================>....] - ETA: 26s - loss: 0.3867 - acc: 0.8632

391/390 [==============================] - 274s 700ms/step - loss: 0.3882 - acc: 0.8629 - val_loss: 1.1716 - val_acc: 0.7417

Epoch 00042: saving model to May16-weights-42-0.74.hdf5
Epoch 43/50
350/390 [=========================>....] - ETA: 26s - loss: 0.3816 - acc: 0.8656

391/390 [==============================] - 273s 699ms/step - loss: 0.3833 - acc: 0.8655 - val_loss: 0.5867 - val_acc: 0.8386

Epoch 00043: saving model to May16-weights-43-0.84.hdf5
Epoch 44/50
350/390 [=========================>....] - ETA: 26s - loss: 0.3778 - acc: 0.8676

391/390 [==============================] - 274s 700ms/step - loss: 0.3810 - acc: 0.8666 - val_loss: 0.4896 - val_acc: 0.8571

Epoch 00044: saving model to May16-weights-44-0.86.hdf5
Epoch 45/50
104/390 [======>.......................] - ETA: 3:07 - loss: 0.3669 - acc: 0.8713

In [23]:
!ls

May16-weights-01-0.48.hdf5  May16-weights-24-0.76.hdf5
May16-weights-02-0.42.hdf5  May16-weights-25-0.77.hdf5
May16-weights-03-0.57.hdf5  May16-weights-26-0.76.hdf5
May16-weights-04-0.41.hdf5  May16-weights-27-0.81.hdf5
May16-weights-05-0.58.hdf5  May16-weights-28-0.74.hdf5
May16-weights-06-0.50.hdf5  May16-weights-29-0.81.hdf5
May16-weights-07-0.67.hdf5  May16-weights-30-0.80.hdf5
May16-weights-08-0.47.hdf5  May16-weights-31-0.77.hdf5
May16-weights-09-0.70.hdf5  May16-weights-32-0.80.hdf5
May16-weights-10-0.58.hdf5  May16-weights-33-0.76.hdf5
May16-weights-11-0.70.hdf5  May16-weights-34-0.78.hdf5
May16-weights-12-0.70.hdf5  May16-weights-35-0.75.hdf5
May16-weights-13-0.60.hdf5  May16-weights-36-0.81.hdf5
May16-weights-14-0.71.hdf5  May16-weights-37-0.80.hdf5
May16-weights-15-0.72.hdf5  May16-weights-38-0.79.hdf5
May16-weights-16-0.76.hdf5  May16-weights-39-0.80.hdf5
May16-weights-17-0.76.hdf5  May16-weights-40-0.83.hdf5
May16-weights-18-0.78.hdf5  May16-weights-41-0.79.hdf5
May16-weig

In [0]:
model.save('May16-weights-45-0.82.hdf5')

from keras.models import load_model
model=load_model('May16-weights-45-0.82.hdf5')

In [25]:

model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                    steps_per_epoch=x_train.shape[0] / batch_size,epochs=5,verbose=1,validation_data=(x_test,y_test),callbacks=callbacks_list)

Epoch 1/5
391/390 [==============================] - 284s 726ms/step - loss: 0.3724 - acc: 0.8702 - val_loss: 0.4734 - val_acc: 0.8638

Epoch 00001: saving model to May16-weights-01-0.86.hdf5
Epoch 2/5
226/390 [================>.............] - ETA: 1:48 - loss: 0.3698 - acc: 0.8703

391/390 [==============================] - 275s 703ms/step - loss: 0.3711 - acc: 0.8697 - val_loss: 0.4725 - val_acc: 0.8559

Epoch 00002: saving model to May16-weights-02-0.86.hdf5
Epoch 3/5
315/390 [=======================>......] - ETA: 49s - loss: 0.3693 - acc: 0.8729

391/390 [==============================] - 275s 704ms/step - loss: 0.3683 - acc: 0.8729 - val_loss: 0.6006 - val_acc: 0.8394

Epoch 00003: saving model to May16-weights-03-0.84.hdf5
Epoch 4/5
342/390 [=========================>....] - ETA: 32s - loss: 0.3587 - acc: 0.8749

391/390 [==============================] - 276s 705ms/step - loss: 0.3592 - acc: 0.8750 - val_loss: 0.6668 - val_acc: 0.8385

Epoch 00004: saving model to May16-weights-04-0.84.hdf5
Epoch 5/5
350/390 [=========================>....] - ETA: 26s - loss: 0.3595 - acc: 0.8745

391/390 [==============================] - 275s 704ms/step - loss: 0.3597 - acc: 0.8744 - val_loss: 0.5768 - val_acc: 0.8500

Epoch 00005: saving model to May16-weights-05-0.85.hdf5


In [26]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 24s 2ms/step
Test loss: 0.5768085919499397
Test accuracy: 0.85


In [0]:
model.save('May16-weights-01-0.86.hdf5')

from keras.models import load_model
model=load_model('May16-weights-01-0.86.hdf5')

In [28]:
# Test the model
score = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

10000/10000 [==============================] - 26s 3ms/step
Test loss: 0.5768085919499397
Test accuracy: 0.85


In [0]:
# Save the trained weights in to .h5 format
model.save_weights("DNST_model.h5")
print("Saved model to disk")

Saved model to disk


In [0]:
from google.colab import files

files.download('DNST_model.h5')